In [1]:
import os

os.environ["SEED"] = "0"
os.environ["TRAIN_DIR"] = "TRAIN"

import numpy as np
from tqdm.auto import tqdm

from src.aac.AAC import AAC
from src.aac.SurrogateEstimator import Estimator1
from src.constant import DATA_DIR, DATABASE_DIR
from src.database import DB
from src.database.queries import get_model_training_data
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_from_index_file
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.log import logger

In [2]:
train_instances = TSP_from_index_file(
    filepath=DATA_DIR / "TSP" / "TRAIN" / "index.json",
    cut_off_cost=100,
    cut_off_time=10,
    n=5,
)
test_instances = TSP_from_index_file(
    filepath=DATA_DIR / "TSP" / "TEST" / "index.json",
    cut_off_cost=100,
    cut_off_time=10,
    n=250,
)

In [5]:
train_instances[0].cut_off_time

10

In [6]:
SOLVERS_N = 2
ATTEMPTS_N = 3

solvers = []
largest_marginal_contribution_solver = None

for solver_i in range(SOLVERS_N):
    logger.info(f"Solver {solver_i + 1}/{SOLVERS_N}")

    best_cost = np.inf
    best_solver = None
    attempt_solvers = []

    for attempt_i in range(ATTEMPTS_N):
        logger.info(f"Attempt {attempt_i + 1}/{ATTEMPTS_N}")

        if largest_marginal_contribution_solver is not None:
            new_solver = largest_marginal_contribution_solver.copy()
        else:
            new_solver = TSP_LKH_Solver()

        iteration_solvers = solvers + [new_solver]

        portfolio = Portfolio.from_iterable(iteration_solvers)
        aac = AAC(
            portfolio=portfolio,
            instance_list=train_instances,
            prefix=f"config;solver={solver_i};attempt={attempt_i}",
            max_iter=2,
            i=solver_i,
            calculate_features=False,
            estimator=None,
        )
        aac.configure()
        result = portfolio.evaluate(
            instance_list=train_instances,
            prefix=f"validate;solver={solver_i};attempt={attempt_i}",
            cache=True,
        )
        attempt_solvers.append(portfolio[solver_i])
        logger.info(f"Attempt {attempt_i + 1}/{ATTEMPTS_N}: cost = {result.cost:.2f}")
        if result.cost < best_cost:
            best_cost = result.cost
            best_solver = portfolio[solver_i]

    solvers.append(best_solver)
    logger.info(f"Solver {solver_i + 1}/{SOLVERS_N}: best cost = {best_cost:.2f}")

    if solver_i < SOLVERS_N - 1:
        largest_marginal_contribution_solver = None
        best_cost = np.inf
        for solver in attempt_solvers:
            if solver != best_solver:
                portfolio = Portfolio.from_iterable(solvers + [solver])
                result = portfolio.evaluate(
                    instance_list=train_instances,
                    prefix=f"largest_marginal_contribution;solver={solver.id()}",
                    cache=True,
                )
                if result.cost < best_cost:
                    best_cost = result.cost
                    largest_marginal_contribution_solver = solver

portfolio = Portfolio.from_iterable(solvers)

[2025-04-27 22:06:10] INFO      Solver 1/2
[2025-04-27 22:06:10] INFO      Attempt 1/3
[2025-04-27 22:06:10] DEBUG     AAC(prefix=config;solver=0;attempt=0, iter=1, configuration_time=[inf])
[2025-04-27 22:06:10] DEBUG     Portfolio.evaluate(config;solver=0;attempt=0;aac_iter=1)
[2025-04-27 22:06:10] DEBUG     Portfolio(size=1)[Solver(id=473831520656465062)]
[2025-04-27 22:06:10] DEBUG     solve(prefix=config;solver=0;attempt=0;aac_iter=1, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/TRAIN/cluster_netgen/000.tsp))
[2025-04-27 22:06:10] DEBUG     solve(prefix=config;solver=0;attempt=0;aac_iter=1, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/TRAIN/compression/000.tsp))
[2025-04-27 22:06:10] DEBUG     solve(prefix=config;solver=0;attempt=0;aac_iter=1, solver=Solver(id=473831520656465062), instance=TSP_Instance(filepath=TSP/TRAIN/expansion/000.tsp))
[2025-04-27 22:06:10] DEBUG     solve(prefix=config;solver=0;attempt=0;aac_iter=1, sol

In [7]:
portfolio

Portfolio(size=2)[Solver(id=85558984741464718), Solver(id=287011037061967493)]

In [30]:
# DB().get_results()